In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import warnings
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
root = '/content/drive/MyDrive/cats_dogs'
train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform=train_transform)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform=test_transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

class_names = train_data.classes
num_classes = len(class_names)

print(f"Detected Classes: {class_names}")
print(f"Training Images: {len(train_data)}")
print(f"Testing Images:  {len(test_data)}")

In [ ]:
AlexNetmodel = models.alexnet(pretrained=True)

for param in AlexNetmodel.features.parameters():
    param.requires_grad = False

torch.manual_seed(42)
AlexNetmodel.classifier = nn.Sequential(
    nn.Linear(9216, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, num_classes),
    nn.LogSoftmax(dim=1)
)

criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(AlexNetmodel.classifier.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AlexNetmodel = AlexNetmodel.to(device)

In [ ]:
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

def show_sample(data_loader):
    images, labels = next(iter(data_loader))
    image = images[0]
    label = labels[0]
    
    image = inv_normalize(image)
    image = image.permute(1, 2, 0).numpy()
    plt.imshow(image)
    plt.title(f"Label: {class_names[label]}")
    plt.axis('off')
    plt.show()

show_sample(train_loader)

In [ ]:
# Training
epochs = 5
AlexNetmodel.train()
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = AlexNetmodel(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# Evaluation and Confusion Matrix
AlexNetmodel.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = AlexNetmodel(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues')
plt.title("Confusion Matrix")
plt.show()